In [ ]:
!pip install -q playwright bs4 nest_asyncio tldextract
!playwright install chromium -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 7.6 MB/s eta 0:00:00
error: unknown option '-q'


#STEP 1: Config

In [ ]:
import os, re, urllib.parse, asyncio
from datetime import datetime, timezone, timedelta
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
import nest_asyncio, tldextract

nest_asyncio.apply()

BASE_URL = "https://www.agnoshealth.com/forums"
RAW_DIR = "/content/drive/MyDrive/agnos-rag/data/raw"
os.makedirs(RAW_DIR, exist_ok=True)

def slugify(text, maxlen=60):
    text = re.sub(r"\s+", "-", text.strip())
    text = re.sub(r"[^A-Za-z0-9\-]+", "", text)
    text = re.sub(r"-{2,}", "-", text).strip("-")
    return text[:maxlen] if text else "untitled"

def abs_url(base, href):
    return urllib.parse.urljoin(base, href) if href else None

def now_iso_bkk():
    return datetime.now(timezone(timedelta(hours=7))).isoformat()


#STEP 2: Scrape forum auto-update

In [ ]:
!playwright install

Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libavif.so.13                                    ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.12/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:269:9)
    at process.processTicksAndRejections (node:internal/process/task_queues:105

In [ ]:
async def scrape_forum_update(max_threads=5):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page(user_agent="Mozilla/5.0 (Colab/Playwright)")

        # ไปที่หน้า forum home
        await page.goto(BASE_URL, wait_until="networkidle")
        home_html = await page.content()

        ts = datetime.now().strftime("%Y%m%d-%H%M%S")
        home_path = os.path.join(RAW_DIR, f"home_{ts}.html")
        with open(home_path, "w", encoding="utf-8") as f:
            f.write(home_html)
        print(f" Saved forum home: {home_path}")

        # หา links
        soup = BeautifulSoup(home_html, "html.parser")
        anchors = soup.find_all("a")
        candidates = []
        seen = set()
        for a in anchors:
            text = (a.get_text(strip=True) or "")
            href = a.get("href")
            if not href: continue
            url = abs_url(BASE_URL, href)
            if not url: continue
            if url in seen: continue
            if "/forums/" in url or "/thread" in url:
                seen.add(url)
                candidates.append((text, url))

        threads = candidates[:max_threads]

        for i, (title, url) in enumerate(threads, 1):
            await page.goto(url, wait_until="networkidle")
            html = await page.content()
            fname = f"thread_{i:04d}_{slugify(title)}_{ts}.html"
            fpath = os.path.join(RAW_DIR, fname)

            meta = (
                "<!--\nMETA:\n"
                f"source_url: {url}\n"
                f"title: {title}\n"
                f"scraped_at: {now_iso_bkk()}\n"
                "-->\n"
            )

            with open(fpath, "w", encoding="utf-8") as f:
                f.write(meta + html)

            print(f"Saved thread {i}: {fpath}")

        await browser.close()

await scrape_forum_update(max_threads=5)

 Saved forum home: /content/drive/MyDrive/agnos-rag/data/raw/home_20250917-154704.html
Saved thread 1: /content/drive/MyDrive/agnos-rag/data/raw/thread_0001_20-Acute-pericarditis72520222-15-8_20250917-154704.html
Saved thread 2: /content/drive/MyDrive/agnos-rag/data/raw/thread_0002_24-Dermatitis-unspecified7222022-5_20250917-154704.html
Saved thread 3: /content/drive/MyDrive/agnos-rag/data/raw/thread_0003_23-Major-depressive-disorder21820243-2-3-5_20250917-154704.html
Saved thread 4: /content/drive/MyDrive/agnos-rag/data/raw/thread_0004_20-Menieres-disease82220222-4-5-2_20250917-154704.html
Saved thread 5: /content/drive/MyDrive/agnos-rag/data/raw/thread_0005_21-Cystitis21620241-2_20250917-154704.html


#เชื่อมกับ 02

In [ ]:
!rm -rf /content/drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
!ls -la "/content/drive/MyDrive/agnos-rag/code"


total 4
-rw------- 1 root root 3516 Sep 17 16:02 ingest_build_index.py


In [ ]:
import importlib.util, os

MODULE_PATH = "/content/drive/MyDrive/agnos-rag/code/ingest_build_index.py"
assert os.path.exists(MODULE_PATH), f"ไม่พบไฟล์: {MODULE_PATH}"

spec = importlib.util.spec_from_file_location("ingest_build_index", MODULE_PATH)
mod = importlib.util.module_from_spec(spec)
spec.loader.exec_module(mod)

build_index_from_html = mod.build_index_from_html
print(" Loaded:", build_index_from_html)

RAW_DIR   = "/content/drive/MyDrive/agnos-rag/data/raw"
INDEX_DIR = "/content/drive/MyDrive/agnos-rag/index"
os.makedirs(INDEX_DIR, exist_ok=True)

build_index_from_html(RAW_DIR, INDEX_DIR)


 Loaded: <function build_index_from_html at 0x7c6398684360>


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

 Build index เสร็จ | posts=71 | chunks=71
   -> /content/drive/MyDrive/agnos-rag/index/faiss.index
   -> /content/drive/MyDrive/agnos-rag/index/meta.parquet


In [ ]:
import pandas as pd, faiss
meta_path = "/content/drive/MyDrive/agnos-rag/index/meta.parquet"
faiss_path = "/content/drive/MyDrive/agnos-rag/index/faiss.index"

meta = pd.read_parquet(meta_path)
idx  = faiss.read_index(faiss_path)

print("rows(meta) =", len(meta), " | vectors(index) =", idx.ntotal)
meta.head(3)


rows(meta) = 71  | vectors(index) = 71


,doc_id,chunk_id,title,url,author,date,content_chunk
0,0,0,หญิง|อายุ20ปีเยื่อหุ้มหัวใจอักเสบ (Acute peric...,https://www.agnoshealth.com/forums/%E0%B9%80%E...,None,7/25/2022,หญิง | อายุ 20 ปี เยื่อหุ้มหัวใจอักเสบ (Acute ...
1,1,0,หญิง|อายุ20ปีเยื่อหุ้มหัวใจอักเสบ (Acute peric...,https://www.agnoshealth.com/forums/%E0%B9%80%E...,None,11/26/2022,หญิง | อายุ 16 ปี เยื่อหุ้มหัวใจอักเสบ (Acute ...
2,2,0,หญิง|อายุ20ปีเยื่อหุ้มหัวใจอักเสบ (Acute peric...,https://www.agnoshealth.com/forums/%E0%B9%80%E...,None,8/9/2022,ชาย | อายุ 30 ปี เยื่อหุ้มหัวใจอักเสบ (Acute p...
